In [26]:
import pandas as pd
import numpy as np
import sqlite3

In [27]:
import tempfile
import os
baza = os.path.join(tempfile.mkdtemp(), "baza.db")
if os.path.isfile(baza):
    os.remove(baza)
conn = sqlite3.connect(baza)

In [28]:
Posts = pd.read_csv("./Posts.csv.gz")
Badges = pd.read_csv("./Badges.csv.gz")
Comments = pd.read_csv("./Comments.csv.gz")
Tags = pd.read_csv("./Tags.csv.gz")
Users = pd.read_csv("./Users.csv.gz")
Votes = pd.read_csv("./Votes.csv.gz")
PostLinks = pd.read_csv("./PostLinks.csv.gz")

In [29]:
Posts.to_sql("Posts", conn)
Badges.to_sql("Badges", conn)
Comments.to_sql("Comments", conn)
Tags.to_sql("Tags", conn)
Users.to_sql("Users", conn)
Votes.to_sql("Votes", conn)
PostLinks.to_sql("PostLinks",conn)

In [44]:
'''Celem jest znalezienie użytkowników,
którzy uzyskali najwięcej Likes, 
wyswietlamy ich dane wraz z najwyżej ocenionym pytaniem.'''
zad1 = pd.read_sql_query(
    """
    SELECT
Users.DisplayName,
Users.Age,
Users.Location,
SUM(Posts.FavoriteCount) AS FavoriteTotal,
Posts.Title AS MostFavoriteQuestion,
MAX(Posts.FavoriteCount) AS MostFavoriteQuestionLikes
FROM Posts
JOIN Users ON Users.Id=Posts.OwnerUserId
WHERE Posts.PostTypeId=1
GROUP BY OwnerUserId
ORDER BY FavoriteTotal DESC
LIMIT 10
    """, conn)

In [45]:

# Join ramek po Id
tmp = pd.merge(Posts,Users,left_on="OwnerUserId",right_on="Id")
# Posty z TypeId == 1 
tmp = tmp.loc[tmp.PostTypeId==1]
# Wybór odpowiednich kolumn
tmp = tmp[["DisplayName","Age","Location","FavoriteCount","Title","OwnerUserId"]]
# przygotowanie danych do dalszej obrobki(agregujac po OwnerUserId)
tmpFO = tmp.groupby("OwnerUserId",sort=True).agg({"FavoriteCount":[np.max,np.sum]}).reset_index(drop=False)
#gubimy MultiIndex
tmpFO.columns =tmpFO.columns.droplevel()
#odpowiednie kolumny
tmpFO.columns = ["OwnerUserId","amax","amin"]
# łączenie z ramką tmp 
tmp = pd.merge(tmp,tmpFO,left_on=["OwnerUserId","FavoriteCount"],right_on=["OwnerUserId","amax"],sort=True)
# przemianowanie kolumn na pasujace do tego co mial SQL
tmp =tmp.rename(columns = {'Title':'MostFavoriteQuestion'})
tmp =tmp.rename(columns = {'amax':'MostFavoriteQuestionLikes'})
tmp =tmp.rename(columns = {'amin':'FavoriteTotal'})
#sortowanko po Favourtie Total
tmp.sort_values(by=['FavoriteTotal'],inplace=True,ascending=False)
#wybranie odpiewiednich kolumn
tmp = tmp.iloc[:, lambda tmp: [0,1,2,7,4,6]]
tmp = tmp.reset_index(drop=True).head(10)
tmp.equals(zad1)


True

In [32]:
'''
Znajdujemy pytania o najwiekszej licznie pozytywnie ocenionych odpowied
'''
zad2 = pd.read_sql_query(
    """
SELECT
Posts.ID,
Posts.Title,
Posts2.PositiveAnswerCount
FROM Posts
JOIN (
SELECT
Posts.ParentID,
COUNT(*) AS PositiveAnswerCount
FROM Posts
WHERE Posts.PostTypeID=2 AND Posts.Score>0
GROUP BY Posts.ParentID
) AS Posts2
ON Posts.ID=Posts2.ParentID
ORDER BY Posts2.PositiveAnswerCount DESC
LIMIT 10
    """, conn)

In [33]:
#odpowiednie query
pos = Posts.query("PostTypeId==2 and Score>0")
#grupuje po ParentId
pos = pos.groupby(["ParentId"],sort=True).size().reset_index()
#odpowiednie kolumny wybieram
pos.columns = ["ParentId","PositiveAnswerCount"]
#ramka pomocnicza
tmp = Posts[["Id","Title"]]
#merguje to co stworzylem do tej pory
wynik = pd.merge(tmp,pos,left_on="Id",right_on = "ParentId")
#sortowanko jak SQL
wynik.sort_values(by=["PositiveAnswerCount",'Id'],inplace=True,ascending=[False,True])
#wybor odpowiednich kolumn
wynik = wynik.iloc[:,lambda wynik:[0,1,3]].reset_index(drop=True)
#najwyzsze 10
wynik = wynik.head(10)
wynik.equals(zad2)

True

In [34]:
'''Znajdujemy pytania, które w danym roku otrzymały najwięcej UpVotes.
'''
zad3 = pd.read_sql_query(
    """
SELECT
Posts.Title,
UpVotesPerYear.Year,
MAX(UpVotesPerYear.Count) AS Count
FROM (
SELECT
PostId,
COUNT(*) AS Count,
STRFTIME('%Y', Votes.CreationDate) AS Year
FROM Votes
WHERE VoteTypeId=2
GROUP BY PostId, Year
) AS UpVotesPerYear
JOIN Posts ON Posts.Id=UpVotesPerYear.PostId
WHERE Posts.PostTypeId=1
GROUP BY Year
    """, conn)

In [35]:
# ramka pomocnicza
upvot = Votes.query("VoteTypeId == 2")
# wybieram rok
upvot.loc[:,"CreationDate"] = upvot.CreationDate.str.slice(0, 4)
# grupowanie po Year 
upvot = upvot.groupby(["PostId","CreationDate"]).size().reset_index()
upvot.columns = ["PostId","Year","Count"]
# nowa ramka z postow tam gdzie PostTypeId ==1
tmp = Posts.query("PostTypeId == 1 ")
tmp = tmp[["Title","Id"]]
#merge ramek
wynik = pd.merge(tmp,upvot,left_on="Id",right_on="PostId")
# agregacja po roku
wynik_tmp = wynik.groupby("Year").agg({"Count":np.max}).reset_index()
# ostateczne polaczenie ramek
wynik = pd.merge(wynik,wynik_tmp,on=["Year","Count"])
# wybieram kolumny 
wynik = wynik.iloc[:,lambda tmp:[0,3,4]]
wynik.equals(zad3)

True

In [36]:
'''
Znajdujemy pytania, gdzie wystąpiła największa różnica pomiędzy najwyżej oceniona odpowiedzią, a odpowiedzią oznaczoną jako 
Accepted Answer.'''
zad4 = pd.read_sql_query(
    """
SELECT
Questions.Id,
Questions.Title,
BestAnswers.MaxScore,
Posts.Score AS AcceptedScore,
BestAnswers.MaxScore-Posts.Score AS Difference
FROM (
SELECT Id, ParentId, MAX(Score) AS MaxScore
FROM Posts
WHERE PostTypeId==2
GROUP BY ParentId
) AS BestAnswers
JOIN (
SELECT * FROM Posts
WHERE PostTypeId==1
) AS Questions
ON Questions.Id=BestAnswers.ParentId
JOIN Posts ON Questions.AcceptedAnswerId=Posts.Id
WHERE Difference>50
ORDER BY Difference DESC
    """, conn)

In [37]:
#przygotowanie danych
pd4 = Posts.query("PostTypeId==1")
tmp = Posts.query("PostTypeId==2")
#grupowanie po ParentId
tmp_1 = tmp.groupby("ParentId").agg({"Score":np.max}).reset_index()
#merge wczesniej zrobionych ramej
tmp = pd.merge(tmp[["ParentId","Id","Score"]],tmp_1,on=['ParentId',"Score"])
#odpowiednie nazwy
tmp = tmp.rename(columns={"Score":"MaxScore"})
#kolejne merge
wynik = pd.merge(pd4,tmp,left_on="Id",right_on="ParentId")
wynik = pd.merge(wynik,Posts,left_on="AcceptedAnswerId",right_on="Id")
#wybranie dancyh wlasciwych
wynik = wynik.loc[:,lambda wynik:["Id_x","Title_x","MaxScore","Score_y"]]
#nowa kolumna Diffrence
wynik["Diffrence"] = wynik["MaxScore"] - wynik["Score_y"]
#przefiltrowanie tam gdzie Diffrence >50
wynik = wynik.query("Diffrence>50")
#posortowanie po Diffrence
wynik.sort_values(by="Diffrence",ascending=False,inplace=True)
wynik = wynik.reset_index(drop=True)
#aby nazwy sie zgadzaly do outputu
wynik.columns = ["Id","Title","MaxScore","AcceptedScore","Difference"]
wynik.equals(zad4)

True

In [38]:
'''
Otrzymujemy pytania do których komentarze dodane przez samego pytającego miały sumarycznie najwyższą ocenę.
'''
pd5 = pd.read_sql_query(
    """
SELECT
Posts.Title,
CmtTotScr.CommentsTotalScore
FROM (
SELECT
PostID,
UserID,
SUM(Score) AS CommentsTotalScore
FROM Comments
GROUP BY PostID, UserID
) AS CmtTotScr
JOIN Posts ON Posts.ID=CmtTotScr.PostID AND Posts.OwnerUserId=CmtTotScr.UserID
WHERE Posts.PostTypeId=1
ORDER BY CmtTotScr.CommentsTotalScore DESC
LIMIT 10
    """, conn)

In [39]:
#odpowiednie pogrupowanie
cmt = Comments.groupby(["PostId","UserId"]).agg({"Score":np.sum}).reset_index()
#wybranie dokladnych tego gdzie PostTypeId ==1
pos = Posts.query("PostTypeId==1")
#polaczenie ramek
cmt = pd.merge(cmt,pos, left_on=["PostId","UserId"],right_on=["Id","OwnerUserId"])
#sortowanie tak samo jak SQl
cmt.sort_values(by=["Score_x","PostId"],ascending=[False,True],inplace=True)
#wybranie docelowych kolumn
cmt = cmt[["Title","Score_x"]]
#sortowanie wyniku
cmt.sort_values(by=["Score_x"],ascending=False,inplace=True)
#wybranie 10 z gory
cmt = cmt.head(10).reset_index(drop=True)
cmt.columns = ["Title","CommentsTotalScore"]
cmt.equals(pd5)

True

In [40]:
'''Informacje o użytkownikach, którym odznakę klasy pierwszej przyznano od 2 do 10 razy.
'''
pd6 = pd.read_sql_query(
"""
SELECT DISTINCT
Users.Id,
Users.DisplayName,
Users.Reputation,
Users.Age,
Users.Location
FROM (
SELECT
Name, UserID
FROM Badges
WHERE Name IN (
SELECT
Name
FROM Badges
WHERE Class=1
GROUP BY Name
HAVING COUNT(*) BETWEEN 2 AND 10
)
AND Class=1
) AS ValuableBadges
JOIN Users ON ValuableBadges.UserId=Users.Id
""",conn)

In [41]:
# Ramka pomocnicza
wyn = Badges.query("Class==1")
# grupuje po kolumnie Name
wyn = wyn.groupby("Name").size().reset_index()
wyn = wyn.rename(columns={0:"Count"})
wyn =wyn.query("10>=Count>=2")

# lista z opserwcjami z Name 
wyn = wyn["Name"].tolist()
# odpowiednie wiersze z Badges
bad = Badges[Badges.Name.isin(wyn)]
bad = bad.query("Class==1")
bad = bad[["Name","UserId"]]
# merge z Users
wynik = pd.merge(bad,Users,left_on="UserId",right_on="Id")
wynik = wynik.iloc[:,lambda tmp:[7,5,10,3,9]].drop_duplicates().reset_index(drop=True)
wynik.equals(pd6)

True

In [42]:
'''Otrzymujemy pytania, które przed 2016 otrzymały najwięcej UpVotes.
'''
pd7 = pd.read_sql_query(
"""
SELECT
Posts.Title,
VotesByAge2.OldVotes
FROM Posts
JOIN (
SELECT
PostId,
MAX(CASE WHEN VoteDate = 'new' THEN Total ELSE 0 END) NewVotes,
MAX(CASE WHEN VoteDate = 'old' THEN Total ELSE 0 END) OldVotes,
SUM(Total) AS Votes
FROM (
SELECT
PostId,
CASE STRFTIME('%Y', CreationDate)
WHEN '2017' THEN 'new'
WHEN '2016' THEN 'new'
ELSE 'old'
END VoteDate,
COUNT(*) AS Total
FROM Votes
WHERE VoteTypeId=2
GROUP BY PostId, VoteDate
) AS VotesByAge
GROUP BY VotesByAge.PostId
HAVING NewVotes=0
) AS VotesByAge2 ON VotesByAge2.PostId=Posts.ID
WHERE Posts.PostTypeId=1
ORDER BY VotesByAge2.OldVotes DESC
LIMIT 10
""",conn)

In [43]:
#wybieramy odpowiednie dane
zd7 = Votes.query("VoteTypeId==2")
#data
zd7.loc[:,"CreationDate"] = zd7.CreationDate.str.slice(0, 4)
#odpowiednia zmiana danych na new albo old
zd7.loc[:,'CreationDate'] = np.where((zd7['CreationDate']=='2017')|(zd7["CreationDate"]=='2016'), 'new', 'old')
#zliczenie danych
zd7 = zd7.groupby(["PostId","CreationDate"]).size().reset_index()
#nowe nazwy kolumn
zd7.columns = ["PostId","VoteDate","Total"]
#odpowiednie zastapienie danych
zd7["NewVotes"] = np.where(zd7["VoteDate"]=="new",zd7["Total"],0)
zd7["OldVotes"] = np.where(zd7["VoteDate"]=="old",zd7["Total"],0)
#pogrupowanie po PostId
zd7 = zd7.groupby("PostId").agg({"Total":np.sum,"NewVotes":np.max,"OldVotes":np.max}).reset_index()
#wybor tego gdzie NewVotes==0
zd7 = zd7.query("NewVotes==0").reset_index(drop=True)
pos = Posts.query("PostTypeId==1")
#merge odpowiednich postow z tym co mialem wczesniej
wynik = pd.merge(zd7,pos,left_on="PostId",right_on="Id")
#posortowanie wartosci
wynik.sort_values(by=["OldVotes","PostId"],ascending=[False,True],inplace=True)
#wybranie odpowiednich kolumn
wynik = wynik.loc[:,lambda wynik:["Title","OldVotes"]]
#ostateczne sortowanie
wynik.sort_values(by="OldVotes",ascending=False,inplace=True)
wynik = wynik.head(10).reset_index(drop=True)
wynik.equals(pd7)


True